In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import math
from timeit import default_timer as timer
from datetime import datetime, timedelta
import numba
import joblib 


/Users/neilkloot/anaconda/lib/python3.6/site-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500
%load_ext autoreload
%autoreload 2


In [3]:
def load(filename):
    """Loads a compressed object from disk
    """
    model_object = joblib.load(filename)
    return model_object


In [4]:
all_df = pd.read_pickle('../data/ml-ta-all-data-201908262001.pkl.gz', compression='gzip')
print('Data loaded')
print(all_df.shape)

Data loaded
(1079523, 1227)


In [5]:
from optimise_dataframe import optimise_df

all_df = optimise_df(all_df)
train_df = all_df

In [6]:
del all_df

In [7]:
target_col = 'target'

numeric_cols = []
category_cols = []

for col in train_df.columns:
    if train_df[col].dtype.name == 'object':
        print(col, train_df[col].dtype.name)
    elif train_df[col].dtype.name == 'category' and col != target_col:
        category_cols.append(col)
        print('category:', col)
    elif train_df[col].dtype.name != 'int8' and col != target_col:
        numeric_cols.append(col)
    elif col == target_col:
        print('--- Target', col, train_df[col].dtype.name)

# print(numeric_cols)


category: one_week_bollinger_prediction
category: two_week_bollinger_prediction
category: four_week_bollinger_prediction
category: eight_week_bollinger_prediction
category: twelve_week_bollinger_prediction
category: twenty_six_week_bollinger_prediction
category: fifty_two_week_bollinger_prediction
category: one_week_bollinger_type
category: two_week_bollinger_type
category: four_week_bollinger_type
category: eight_week_bollinger_type
category: twelve_week_bollinger_type
category: twenty_six_week_bollinger_type
category: fifty_two_week_bollinger_type
category: symbol
category: GICSSector
category: GICSIndustryGroup
category: GICSIndustry
--- Target target float32


In [8]:
# Split data for test 
train_df.sort_index(inplace=True)


In [9]:
from sklearn import preprocessing

# Scale all numeric columns excluding target col
print('Scaling', len(numeric_cols), 'of', train_df.shape[1], 'cols')

scaler = preprocessing.MinMaxScaler()

# Fill any infinities, -infinites, NaNs which will cause scaler to fail
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
train_df[numeric_cols] = train_df[numeric_cols].fillna(0)


# Transform data and fill any NAs
scaler.fit(train_df[numeric_cols])
train_df[numeric_cols] = scaler.transform(train_df[numeric_cols])
train_df[numeric_cols] = train_df[numeric_cols].fillna(0)


joblib.dump(scaler, '../models/scaler.joblib.z') 


Scaling 1007 of 1227 cols


/Users/neilkloot/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


['../models/scaler.joblib.z']

In [10]:
train_df[numeric_cols].head(100)

,lastTradePriceOnly,adjustedPrice,volume,previousClose,change,changeInPercent,52WeekHigh,52WeekLow,changeFrom52WeekHigh,changeFrom52WeekLow,percebtChangeFrom52WeekHigh,percentChangeFrom52WeekLow,allordpreviousclose,allordchange,allorddayshigh,allorddayslow,allordpercebtChangeFrom52WeekHigh,allordpercentChangeFrom52WeekLow,asxpreviousclose,asxchange,asxdayshigh,asxdayslow,asxpercebtChangeFrom52WeekHigh,asxpercentChangeFrom52WeekLow,exDividendRelative,exDividendPayout,640106_A3597525W,AINTCOV,Beta,BookValuePerShareYear,CashPerShareYear,DPSRecentYear,EPS,FIRMMCRT,FXRUSD,Float,GRCPAIAD,GRCPAISAD,GRCPBCAD,GRCPBCSAD,GRCPBMAD,GRCPNRAD,GRCPRCAD,H01_GGDPCVGDP,H01_GGDPCVGDPFY,H05_GLFSEPTPOP,MarketCap,OperatingMargin,PE,QuoteLast,ReturnOnEquityYear,TotalDebtToEquityYear,daysHigh,daysLow,one_week_min,one_week_max,one_week_mean,one_week_median,one_week_std,one_week_bollinger_upper,one_week_bollinger_lower,one_week_comparison_adjustedPrice,one_week_price_change,one_week_price_return,one_week_dividend_value,one_week_dividend_return,one_week_total_return,two_week_min,two_week_max,two_week_mean,two_week_median,two_week_std,two_week_bollinger_upper,two_week_bollinger_lower,two_week_comparison_adjustedPrice,two_week_price_change,two_week_price_return,two_week_dividend_value,two_week_dividend_return,two_week_total_return,four_week_min,four_week_max,four_week_mean,four_week_median,four_week_std,four_week_bollinger_upper,four_week_bollinger_lower,four_week_comparison_adjustedPrice,four_week_price_change,four_week_price_return,four_week_dividend_value,four_week_dividend_return,four_week_total_return,eight_week_min,eight_week_max,eight_week_mean,eight_week_median,eight_week_std,eight_week_bollinger_upper,eight_week_bollinger_lower,eight_week_comparison_adjustedPrice,eight_week_price_change,eight_week_price_return,eight_week_dividend_value,eight_week_dividend_return,eight_week_total_return,twelve_week_min,twelve_week_max,twelve_week_mean,twelve_week_median,twelve_week_std,twelve_week_bollinger_upper,twelve_week_bollinger_lower,twelve_week_comparison_adjustedPrice,twelve_week_price_change,twelve_week_price_return,twelve_week_dividend_value,twelve_week_dividend_return,twelve_week_total_return,twenty_six_week_min,twenty_six_week_max,twenty_six_week_mean,twenty_six_week_median,twenty_six_week_std,twenty_six_week_bollinger_upper,twenty_six_week_bollinger_lower,twenty_six_week_comparison_adjustedPrice,twenty_six_week_price_change,twenty_six_week_price_return,twenty_six_week_dividend_value,twenty_six_week_dividend_return,twenty_six_week_total_return,fifty_two_week_min,fifty_two_week_max,fifty_two_week_mean,fifty_two_week_median,fifty_two_week_std,fifty_two_week_bollinger_upper,fifty_two_week_bollinger_lower,fifty_two_week_comparison_adjustedPrice,fifty_two_week_price_change,fifty_two_week_price_return,fifty_two_week_dividend_value,fifty_two_week_dividend_return,fifty_two_week_total_return,asxpreviousclose_T11_20P,asxpreviousclose_T1P,asxpreviousclose_T2_5P,asxpreviousclose_T6_10P,allordpreviousclose_T1P,allordpreviousclose_T2_5P,allordpreviousclose_T6_10P,adjustedPrice_T11_20P,adjustedPrice_T1P,adjustedPrice_T2_5P,adjustedPrice_T6_10P,FIRMMCRT_T11_20P,FIRMMCRT_T1P,FIRMMCRT_T2_5P,FIRMMCRT_T6_10P,FXRUSD_T11_20P,FXRUSD_T1P,FXRUSD_T2_5P,FXRUSD_T6_10P,GRCPAIAD_T11_20P,GRCPAIAD_T1P,GRCPAIAD_T2_5P,GRCPAIAD_T6_10P,GRCPAISAD_T1P,GRCPAISAD_T2_5P,GRCPAISAD_T6_10P,GRCPAISAD_T11_20P,GRCPBCAD_T1P,GRCPBCAD_T2_5P,GRCPBCAD_T6_10P,GRCPBCAD_T11_20P,GRCPBCSAD_T1P,GRCPBCSAD_T2_5P,GRCPBCSAD_T6_10P,GRCPBCSAD_T11_20P,GRCPBMAD_T1P,GRCPBMAD_T2_5P,GRCPBMAD_T6_10P,GRCPBMAD_T11_20P,GRCPNRAD_T1P,GRCPNRAD_T2_5P,GRCPNRAD_T6_10P,GRCPNRAD_T11_20P,GRCPRCAD_T1P,GRCPRCAD_T2_5P,GRCPRCAD_T6_10P,GRCPRCAD_T11_20P,H01_GGDPCVGDPFY_T1P,H01_GGDPCVGDPFY_T2_5P,H01_GGDPCVGDPFY_T6_10P,H01_GGDPCVGDPFY_T11_20P,H05_GLFSEPTPOP_T1P,H05_GLFSEPTPOP_T2_5P,H05_GLFSEPTPOP_T6_10P,H05_GLFSEPTPOP_T11_20P,totalVolume,weekLow,weekHigh,weekOpen,weekClose,ta_volume_adi,ta_volume_obv,ta_volume_cmf,ta_volume_fi,ta_volume_em,

In [11]:
def one_hot_encode_field(df, column_name, categories):
    new_cols = pd.get_dummies(df[column_name])
    new_cols = new_cols.astype('int8', errors='ignore')

    new_cols.T.reindex(categories).T.fillna(0)
    
    name_map = {}
    # rename the categories
    for val in categories:
        name_map[val] = val

    new_cols.rename(name_map, axis=1, inplace=True)
    
    # Remove the original column
    df.drop([column_name], axis=1, inplace=True)
    
    # Return df with new cols 
    return pd.concat([df, new_cols], axis=1)



In [12]:
from sklearn.preprocessing import LabelEncoder

def prep_class_index(df, column_name, categories):
    # Create 
    val_map = {}
    for i in range(len(categories)):
        val_map[categories[i]] = i
        
    print(val_map)
    df['class_index'] = df[column_name].map(val_map).values
    
    # Return df with new target index col
    return df


In [13]:
bins = [-99.07692719, -13.13461361, -3.00000238, 0.,9.25964718, 1493.]
bin_names = ['strong_sell', 'sell', 'hold', 'buy', 'strong_buy']


In [14]:
# prep data for training with one hot encoding
train_df['class'] = pd.cut(train_df['target'], bins=bins,labels=bin_names)
train_df = prep_class_index(train_df, 'class', bin_names)
train_df = one_hot_encode_field(train_df, 'class', bin_names)



{'strong_sell': 0, 'sell': 1, 'hold': 2, 'buy': 3, 'strong_buy': 4}


In [39]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout
from keras.layers.embeddings import Embedding
from keras import optimizers, initializers

# Determine unique vals for each category
col_vals_dict = {c: list(train_df[c].unique()) for c in category_cols}

embed_cols = []
for c in category_cols:
    if len(col_vals_dict[c])>2:
        embed_cols.append(c)
        print(c + ': %d values' % len(col_vals_dict[c])) #look at value counts to know the embedding dimensions

        
# Build embedding network for category cols
inputs = []
embeddings = []

initalizer = initializers.TruncatedNormal(stddev=0.00005, seed=1975)
print('Initializer:', initalizer)

for c in embed_cols:
    embedding_input = Input(shape=(1,))
    vocab_size = len(col_vals_dict[c]) + 1
    embedding_size = int(min(np.ceil(vocab_size/2), 50))
    embedding = Embedding(vocab_size, embedding_size, input_length=1, embeddings_initializer=initalizer)(embedding_input)
    embedding = Reshape(target_shape=(embedding_size,))(embedding)
    print('Create embedding', c, col_vals_dict[c])
    print('Embedding size', embedding_size)
    print('Vocab size', vocab_size)
    inputs.append(embedding_input)
    embeddings.append(embedding)
    
# Add embedding for numeric cols
input_numeric = Input(shape=(len(numeric_cols),))
embedding_numeric = Dense(len(numeric_cols))(input_numeric) 
inputs.append(input_numeric)
embeddings.append(embedding_numeric)
print('Create numeric embedding')
print('Input shape', (len(numeric_cols),))


# Build model
x = Concatenate()(embeddings)
x = Dense(1000, activation='relu', kernel_initializer=initalizer)(x)
x = Dropout(.15)(x)
x = Dense(500, activation='relu', kernel_initializer=initalizer)(x)
x = Dropout(.15)(x)
x = Dense(200, activation='relu', kernel_initializer=initalizer, name='int_layer')(x)
x = Dropout(.15)(x)
output = Dense(5, activation='softmax')(x)

model = Model(inputs, output)
                             
model.summary()

model.compile(loss='kullback_leibler_divergence', optimizer=optimizers.Adamax(learning_rate=0.006),
              metrics=['accuracy'])

intermediate_model = Model(inputs=model.input, outputs=model.get_layer('int_layer').output)


one_week_bollinger_prediction: 3 values
two_week_bollinger_prediction: 3 values
four_week_bollinger_prediction: 3 values
eight_week_bollinger_prediction: 3 values
twelve_week_bollinger_prediction: 3 values
twenty_six_week_bollinger_prediction: 3 values
fifty_two_week_bollinger_prediction: 3 values
one_week_bollinger_type: 3 values
two_week_bollinger_type: 3 values
four_week_bollinger_type: 3 values
eight_week_bollinger_type: 3 values
twelve_week_bollinger_type: 3 values
twenty_six_week_bollinger_type: 3 values
fifty_two_week_bollinger_type: 3 values
symbol: 2632 values
GICSSector: 13 values
GICSIndustryGroup: 26 values
GICSIndustry: 69 values
Initializer: <keras.initializers.TruncatedNormal object at 0x17724b2e8>
Create embedding one_week_bollinger_prediction ['Steady', 'Falling', 'Rising']
Embedding size 2
Vocab size 4
Create embedding two_week_bollinger_prediction ['Steady', 'Rising', 'Falling']
Embedding size 2
Vocab size 4
Create embedding four_week_bollinger_prediction ['Steady', 

Embedding size 35
Vocab size 70
Create numeric embedding
Input shape (1007,)
Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_210 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
input_211 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
input_212 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
input_213 (InputLayer)          (None, 1)            0                                            
______________

In [16]:
# Prep data for model

input_list_train = []
category_val_maps = {}

# the cols to be embedded: rescaling to range [0, # values)
for c in category_cols:
    print('Processing', c)
    # Get full set of possible vals from complete data-set
    raw_vals = np.unique(train_df[c])
    val_map = {}
    for i in range(len(raw_vals)):
        val_map[raw_vals[i]] = i       
    category_val_maps[c] = val_map
    input_list_train.append(train_df[c].map(val_map).values)

#the rest of the columns
print('Processing numeric cols')
print(train_df[numeric_cols].shape)
input_list_train.append(train_df[numeric_cols].values)

print('Train X len', len(input_list_train))
print('Train X nulls', train_df[category_cols].isnull().any().sum())

joblib.dump(category_val_maps, '../models/category_val_maps.joblib.z') 


Processing one_week_bollinger_prediction
Processing two_week_bollinger_prediction
Processing four_week_bollinger_prediction
Processing eight_week_bollinger_prediction
Processing twelve_week_bollinger_prediction
Processing twenty_six_week_bollinger_prediction
Processing fifty_two_week_bollinger_prediction
Processing one_week_bollinger_type
Processing two_week_bollinger_type
Processing four_week_bollinger_type
Processing eight_week_bollinger_type
Processing twelve_week_bollinger_type
Processing twenty_six_week_bollinger_type
Processing fifty_two_week_bollinger_type
Processing symbol
Processing GICSSector
Processing GICSIndustryGroup
Processing GICSIndustry
Processing numeric cols
(1079523, 1007)
Train X len 19
Train X nulls 0


['../models/category_val_maps.joblib.z']

In [17]:
# Set y values
y_train = train_df[bin_names].values
print('Train y shape', y_train.shape)
print('Train y nulls', train_df[bin_names].isnull().any().sum())


Train y shape (1079523, 5)
Train y nulls 0


In [40]:
# Train model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

print('Initializer:', initalizer)

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.00001, verbose=1)
model.fit(input_list_train, 
          y_train, 
          epochs=1000, 
          batch_size=1024,#256, 
          verbose=1, 
          callbacks=[reduce_lr]
         )

# 500, 200, 100 lr 0.004 => epoch 60: loss: 1.2622, accuracy: 0.4701, num lr reductions: 0
# 1000, 500, 200 lr 0.006 => epoch 60: loss: 1.3076, accuracy: 0.4484, num lr reductions: 0
# 1000, 500, 200 lr 0.006 => epoch 1000: loss: 1.0373 , accuracy: 0.5743, num lr reductions: max


Initializer: <keras.initializers.TruncatedNormal object at 0x17724b2e8>


/Users/neilkloot/anaconda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1000
1079523/1079523 [==============================] - 196s 182us/step - loss: 1.4583 - accuracy: 0.3606
Epoch 2/1000
1079523/1079523 [==============================] - 197s 183us/step - loss: 1.3995 - accuracy: 0.4022
Epoch 3/1000
1079523/1079523 [==============================] - 197s 182us/step - loss: 1.3836 - accuracy: 0.4107
Epoch 4/1000
1079523/1079523 [==============================] - 198s 183us/step - loss: 1.3648 - accuracy: 0.4203
Epoch 5/1000
1079523/1079523 [==============================] - 197s 183us/step - loss: 1.3530 - accuracy: 0.4265
Epoch 6/1000
1079523/1079523 [==============================] - 197s 182us/step - loss: 1.3454 - accuracy: 0.4302
Epoch 7/1000
1079523/1079523 [==============================] - 196s 181us/step - loss: 1.3383 - accuracy: 0.4330
Epoch 8/1000
1079523/1079523 [==============================] - 198s 183us/step - loss: 1.3312 - accuracy: 0.4362
Epoch 9/1000
1079523/1079523 [==============================] - 199s 184us/step - loss: 

KeyboardInterrupt: 

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt


def plot_confusion_matrix(cm, labels, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
pred_train_y = model.predict(input_list_train)
pred_test_y = model.predict(input_list_test)

intermediate_train_vals = intermediate_model.predict(input_list_train)
intermediate_test_vals = intermediate_model.predict(input_list_test)

In [ ]:
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

train_class_labels = np.argmax(pred_train_y, axis=1)
train_class_vector_one_hot = to_categorical(train_class_labels)
train_class_actuals = np.argmax(y_train, axis=1)

test_class_labels = np.argmax(pred_test_y, axis=1)
test_class_vector_one_hot = to_categorical(test_class_labels)
test_class_actuals = np.argmax(y_test, axis=1)


print(accuracy_score(class_actuals, class_labels))

print(accuracy_score(test_class_actuals, test_class_labels))



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


print('Scoring validation')
score = accuracy_score(test_class_actuals, test_class_labels)
 
print('Accuracy:', score)

cm = confusion_matrix(test_class_actuals, test_class_labels)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm, bin_names, title='Confusion matrix')

# Normalize the confusion matrix by row (i.e by the number of samples in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, bin_names, title='Normalized confusion matrix')

print('Scoring backtest data')
score = accuracy_score(train_class_actuals, train_class_labels)
 
print('Accuracy:', score)

cm = confusion_matrix(train_class_actuals, train_class_labels)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm, bin_names, title='Confusion matrix')

# Normalize the confusion matrix by row (i.e by the number of samples in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, bin_names, title='Normalized confusion matrix')

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, VotingClassifier

histogram_model = HistGradientBoostingClassifier(
    max_iter=343, 
    max_leaf_nodes=623, 
    learning_rate=0.1, 
    min_samples_leaf=2, 
    l2_regularization=0.,
    verbose=1
)
    
print('-' * 50)
print('Fitting histogram model')
histogram_model.fit(intermediate_train_vals, train_class_actuals)
print('-' * 50)
print('Histogram model done')

hist_train_pred_y = histogram_model.predict(intermediate_train_vals)
hist_test_pred_y = histogram_model.predict(intermediate_test_vals)



In [ ]:
print('Scoring validation')
pred_class_labels = hist_test_pred_y
score = accuracy_score(test_class_actuals, pred_class_labels)
 
print('Accuracy:', score)

cm = confusion_matrix(test_class_actuals, pred_class_labels)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm, bin_names, title='Confusion matrix')

# Normalize the confusion matrix by row (i.e by the number of samples in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, bin_names, title='Normalized confusion matrix')


print('Scoring backtest')
pred_class_labels = hist_train_pred_y
score = accuracy_score(train_class_actuals, pred_class_labels)
 
print('Accuracy:', score)

cm = confusion_matrix(train_class_actuals, pred_class_labels)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm, bin_names, title='Confusion matrix')

# Normalize the confusion matrix by row (i.e by the number of samples in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, bin_names, title='Normalized confusion matrix')


In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model =  RandomForestClassifier(
         max_depth=20, n_estimators=505, min_samples_split=2, min_samples_leaf=1
)

print('-' * 50)
print('Fitting random forest model')
random_forest_model.fit(intermediate_vals, class_actuals)
print('-' * 50)
print('Random forest done')

train_pred_y = random_forest_model.predict(intermediate_vals)

print('Scoring')
pred_class_labels = train_pred_y
score = accuracy_score(class_actuals, pred_class_labels)
 
print('Accuracy:', score)

cm = confusion_matrix(class_actuals, pred_class_labels)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm, bin_names, title='Confusion matrix')

# Normalize the confusion matrix by row (i.e by the number of samples in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, bin_names, title='Normalized confusion matrix')

In [ ]:
from mlxtend.classifier import EnsembleVoteClassifier

voting_model = EnsembleVoteClassifier(
    clfs=[random_forest_model, histogram_model], 
    voting='soft',
    refit=False,
    verbose=1)

print('Fitting voting classifier')
voting_model.fit(intermediate_vals, class_actuals)
print('-' * 50)
print('Voting classifier done')

train_pred_y = voting_model.predict(intermediate_vals)

print('Scoring')
pred_class_labels = train_pred_y
score = accuracy_score(class_actuals, pred_class_labels)
 
print('Accuracy:', score)

cm = confusion_matrix(class_actuals, pred_class_labels)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm, bin_names, title='Confusion matrix')

# Normalize the confusion matrix by row (i.e by the number of samples in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, bin_names, title='Normalized confusion matrix')
